# eidein

Interactive tool for dimensionality reduction of astronomical data and their visualisation.

In [ ]:
%matplotlib widget

from astropy.modeling import models, fitting
import h5py
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np

from eidein import Eidein

In [ ]:
with h5py.File("dataset.hdf5", "r") as datafile:
    ids = datafile["ids"][:]
    X = datafile["X"][:]
    uncertainty = datafile["uncertainty"][:]
    predictions = datafile["predictions"][:]
ids.dtype, X.dtype, uncertainty.dtype, predictions.dtype

def id2filename(plate, mjd, fiberid):
    return "spec-{:04d}-{}-{:04d}.fits".format(plate, mjd, fiberid)

vec_id2filename = np.vectorize(id2filename)
identifiers = vec_id2filename(ids[:, 0], ids[:, 1], ids[:, 2])

In [ ]:
LOGLAMMIN, LOGLAMMAX = 3.5832, 3.9583
N_FEATURES = 3752
LAM = np.power(10, np.linspace(LOGLAMMIN, LOGLAMMAX, N_FEATURES))
LINES = [
    (1033.82, "O VI"),
    (1215.24, "Lyα"),
    (1549.48, "C IV"),
    (1908.734, "C III"),
    (2326.0, "C II"),
    (2799.117, "Mg II"),
    (4102.89, "HΔ"),
    (4341.68, "Hγ"),
    (4862.68, "Hβ"),
    (6564.61, "Hα")]
ARROWPROPS = {"arrowstyle": "-|>", "facecolor": "black"}

def z2lam_emit(z, lam_obsv):
    return lam_obsv / (1 + z)

def plot_function(ax, filename, flux, y, label, lam=LAM):
    label_str = "{}\n$\hat{{z}} = {:.2f}$\n$z = {:.2f}$ (shown)".format(
        filename, y, label)
    
    lam_emit = [z2lam_emit(label, l) for l in lam]
    ax.plot(lam_emit, flux, label=label_str)
    ax.legend()
    ax.set_xlabel("Rest Frame Wavelength [Å]")
    ax.set_ylabel("Flux [10$^{-17}$ erg cm$^{-2}$ s$^{-1}$ Å$^{-1}$]")
    
    # plot spectral lines
    for line, name in LINES:
        ax.annotate(
            name,
            xy=(line, 0), xytext=(line, -2),
            arrowprops=ARROWPROPS,
            horizontalalignment='center')

In [ ]:
label_widget = ipywidgets.FloatText(description="Redshift:", step=0.01)
eidein = Eidein(identifiers, X, predictions, uncertainty, plot_function, label_widget)
display(eidein)

In [ ]:
eidein.labelled